In [1]:
import os
import csv
import sys
import torch

from torch.optim import lr_scheduler
from torch.utils.data import DataLoader

from core.dataset import EDATempDatasetAuxLosses2

from core.optimization import optimize_av_losses
from core.io import set_up_log_and_ws_out
from core.util import configure_backbone_forward_phase, load_train_video_set, load_val_video_set, load_test_video_set

import core.custom_transforms as ct
import core.config as exp_conf

import pickle

In [6]:
clip_lenght = 11
cuda_device_number = '0'

model_weights = './STE_TRAIN/ste_encoder/60.pth'

# the file('results.csv')'s path which is made after forward process
target_directory = './KEMDy20'

csv_root = './KEMDy20'
audio_root = './KEMDy20/wav'

io_config = exp_conf.STE_inputs
opt_config = exp_conf.STE_forward_params
opt_config['batch_size'] = 1

# cuda config
backbone = configure_backbone_forward_phase(opt_config['backbone'], model_weights, clip_lenght)
has_cuda = torch.cuda.is_available()
device = torch.device('cuda:'+cuda_device_number if has_cuda else 'cpu')
backbone = backbone.to(device)

video_data_transforms = {
    'val': ct.video_val
}

In [7]:
pkl_list = list()

with open(target_directory+'results'+'.csv', mode='w') as vf:
    vf_writer = csv.writer(
        vf, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    d_train = EDATempDatasetAuxLosses2(
        audio_root=audio_root, csv_root=csv_root, clip_length=11)

    dl_train = DataLoader(
        d_train, batch_size=opt_config['batch_size'], shuffle=False, num_workers=opt_config['threads'])

    for idx, dl in enumerate(dl_train):
        try:
            print(' \t Forward iter ', idx, '/', len(dl_train), end='\r')
            data, emotion, target_seg, target_ts = dl
            EDAt, TEMPt, audio_data = data

            TEMPt = TEMPt.to(device)
            EDAt = EDAt.to(device)
            audio_data = audio_data.to(device)
            emotion = emotion.to(device)

            with torch.set_grad_enabled(False):
                EDA_temp_audio_out, EDA_out, temp_out, audio_out, feats = backbone(
                    EDAt, TEMPt, audio_data)

                preds = torch.argmax(EDA_temp_audio_out,
                                     1).detach().cpu().numpy()
                gts = torch.argmax(emotion, 1).detach().cpu().numpy()
                emotion = emotion.detach().cpu().numpy()

                feats = feats.detach().cpu().numpy()[0]

                vf_writer.writerow(
                    [target_ts[0], target_seg[0], emotion[0], gts[0], preds[0], list(feats)])

                pkl_list.append([target_ts[0], target_seg[0], emotion[0], gts[0], preds[0], list(feats)])
        except Exception as e:
            print(e)
            continue

KeyboardInterrupt: 

In [ ]:
def custom_ts_to_seconds(td):
    ts = td.split('-')[-3:]

    seconds = 0
    seconds = float(ts[0])//100*3600 + float(ts[0]) % 100 * \
        60 + float(ts[1]) + float(ts[2])*0.001

    return seconds

In [7]:
# pkl_list

In [8]:
pkl_dict = dict()
name_dict = dict()

for it in pkl_list:
    isExist = name_dict.get(it[1], 0)
    new_dict = {
        'ts': custom_ts_to_seconds(it[0]),
        'feature': it[5],
        'label': it[2]
    }

    if isExist:
        pkl_dict[it[1]].append(new_dict)
    else:
        name_dict[it[1]] = 1
        pkl_dict[it[1]] = list()
        pkl_dict[it[1]].append(new_dict)


In [10]:
pkl_dict.keys()

dict_keys(['Sess01_script01_User001F_001', 'Sess01_script01_User001F_002', 'Sess01_script01_User001F_003', 'Sess01_script01_User001F_004', 'Sess01_script01_User001F_005', 'Sess01_script01_User001F_006', 'Sess01_script01_User001F_007', 'Sess01_script01_User001F_008', 'Sess01_script01_User001F_009', 'Sess01_script01_User001F_010', 'Sess01_script01_User001F_011', 'Sess01_script01_User001F_012', 'Sess01_script01_User001F_013', 'Sess01_script01_User001F_014', 'Sess01_script01_User001F_015', 'Sess01_script01_User002M_001', 'Sess01_script01_User002M_002', 'Sess01_script01_User002M_003', 'Sess01_script01_User002M_004', 'Sess01_script01_User002M_005', 'Sess01_script01_User002M_006', 'Sess01_script01_User002M_007', 'Sess01_script01_User002M_008', 'Sess01_script01_User002M_009', 'Sess01_script01_User002M_010', 'Sess01_script01_User002M_011', 'Sess01_script01_User002M_012', 'Sess01_script01_User002M_013', 'Sess01_script01_User002M_014', 'Sess01_script01_User002M_015', 'Sess01_script01_User002M_016

In [12]:
import pickle

In [13]:
with open(f'{csv_root}/feat.pkl','wb') as fw:
    pickle.dump(pkl_dict, fw)

In [15]:
feature_dir = f'{fcsv_root}/feats.pkl'

feature_sessions = {}
with open(feature_dir, 'rb') as fl:
    feature = pickle.load(fl)
    
    for key in feature.keys():
        # value의 형식은 {name: [{timestamp: 0000, feature: [1024], label: [10]}, {timestamp: 0001, ...}], ...}
        values = feature[key]
        
        # name은 Sess01_script01_User002M_001와 같은 형식 ['Sess01', 'script01', 'User002M', '001']
        name = key.split('_')
        session = name[0]
        if session not in feature_sessions:
            feature_sessions[session] = []
        
        for value in values:
            data = []
            data.append(name[0])
            data.append(value['ts'])
            data.append(name[2])
            data.append(value['label'])
            data.append(value['feature'])
            feature_sessions[session].append(data)
        
os.makedirs(f"{csv_root}/Session, exist_ok=True)
            
for session in feature_sessions.keys():
    with open(f"{csv_root}/Session/{}.pkl".format(session), "wb") as fl:
        pickle.dump(feature_sessions[session], fl)